In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Khóa luận tốt nghiệp

/content/drive/.shortcut-targets-by-id/1T-TkLmhLIA8qodciANOIZEcC0W8CsU9V/Khóa luận tốt nghiệp


In [ ]:
!pip install genz-tokenize -q 
!pip install vncorenlp -q 
!pip install fasttext -q 
!pip install pyvi -q

     |████████████████████████████████| 63.9 MB 1.4 MB/s 
     |████████████████████████████████| 4.2 MB 40.5 MB/s 
     |████████████████████████████████| 462 kB 53.9 MB/s 
     |████████████████████████████████| 84 kB 3.0 MB/s 
     |████████████████████████████████| 596 kB 31.3 MB/s 
     |████████████████████████████████| 6.6 MB 40.8 MB/s 
     |████████████████████████████████| 2.6 MB 7.2 MB/s 
     |████████████████████████████████| 68 kB 4.4 MB/s 
     |████████████████████████████████| 8.5 MB 7.6 MB/s 
     |████████████████████████████████| 965 kB 58.8 MB/s 


In [ ]:
from genz_tokenize import Tokenize, get_embedding_matrix
import pandas as pd
import numpy as np
import string
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import fasttext
import tensorflow as tf
import pickle
from vncorenlp import VnCoreNLP
from pyvi import ViTokenizer, ViPosTagger

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/Khóa luận tốt nghiệp/weight_of_word/data2.xlsx', header = None)
data

In [ ]:
x = data.values[:, 0]
y = data.values[:, 1]

In [ ]:
punc = string.punctuation
punc = punc.replace('_', '')
punc = punc.replace('/', '')
punc = punc.replace(',', '')
punc = punc.replace('$', '')
punc = punc.replace('%', '')
def preprocess(text):
    text = [i for i in text if i not in punc]
    text = ''.join(text)
    return text.lower()

In [ ]:
x = [preprocess(i) for i in x]
y = [preprocess(i) for i in y]

In [ ]:
tokenizer = Tokenize()

In [ ]:
# pickle.dump(embedding_matrix, open('weight_of_word/emb.pkl', 'wb'))
embedding_matrix = get_embedding_matrix()

In [ ]:
maxlen = max(277, max([len(i.split()) for i in x]))
maxlen

In [ ]:
def vncore_tokenize(text):
    result = vncore.tokenize(text)
    text = ' '
    for i in result:
        text+= ' '.join(i)+' '
    return text.strip()

In [ ]:
train_y = []
for i, j in zip(x, y):
    s = []
    for c in i.split():
        if c in j:
            a = np.random.uniform(0.8, 0.95)
        else:
            a = np.random.uniform(0.1, 0.3)
        s.append(a)
    s = s+[0]*(maxlen - len(s))
    train_y.append(s)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(x, train_y, test_size = 0.1)

In [ ]:
label = ['ai', 'cai gi', 'con vat', 'nhu the nao', 'number', 'tai sao', 'thoi gian', 'time', 'thuc vat', 'yes no', 'location']
class Embedding(tf.keras.layers.Layer):
    def __init__(self, vocab_size, dim,max_position_embeddings, **kwargs):
        super().__init__(**kwargs)
        self.vocab_size = vocab_size
        self.dim = dim
        self.max_position_embeddings = max_position_embeddings
        self.LayerNorm = tf.keras.layers.LayerNormalization(epsilon=1e-12, name="LayerNorm")
        self.dropout = tf.keras.layers.Dropout(rate=0.1)

    def build(self, input_shape: tf.TensorShape):
        with tf.name_scope("word_embeddings"):
            self.weight = self.add_weight(
                name="weight",
                shape=[self.vocab_size, self.dim],
            )

        with tf.name_scope("position_embeddings"):
            self.position_embeddings = self.add_weight(
                name="embeddings",
                shape=[self.max_position_embeddings, self.dim],
            )
        super().build(input_shape)

    def call(self, input_ids=None):
        inputs_embeds = tf.gather(params=self.weight, indices=input_ids)
        input_shape = tf.shape(input_ids)
        position_ids = tf.expand_dims(tf.range(start=0, limit=input_shape[-1]), axis=0)
        position_embeds = tf.gather(params=self.position_embeddings, indices=position_ids)
        final_embeddings = inputs_embeds + position_embeds
        final_embeddings = self.LayerNorm(inputs=final_embeddings)
        final_embeddings = self.dropout(inputs=final_embeddings)
        return final_embeddings

class FeedFoward(tf.keras.layers.Layer):
    def __init__(self, d_model, num_class):
        super(FeedFoward, self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(d_model)
        self.dropout = tf.keras.layers.Dropout(0.2)
        self.out = tf.keras.layers.Dense(num_class, activation = 'softmax')
    
    def call(self, inputs):
        inputs = self.flatten(inputs)
        x = self.fc1(inputs)
        x = self.dropout(x)
        return self.out(x)
class ClassifyModel(tf.keras.Model):
    def __init__(self, vocab_size, max_position_embeddings, d_model, num_heads, num_class):
        super(ClassifyModel, self).__init__()
        self.embedding = Embedding(vocab_size, d_model, max_position_embeddings)
        self.layernorm = tf.keras.layers.LayerNormalization(epsilon=1e-6)
        self.mha = tf.keras.layers.MultiHeadAttention(num_heads, d_model, dropout = 0.1)
        self.fc = tf.keras.layers.Dense(1)
        self.fw = FeedFoward(d_model, num_class)
    def call(self, inputs):
        x = self.embedding(inputs)
        mask = self.mask(inputs)
        out_mha = self.mha(x, x, x)
        x = self.fc(self.layernorm(x+out_mha))
        return self.fw(x)

    def mask(self, inputs):
        masks = tf.logical_not(tf.math.equal(inputs, 0))
        masks = tf.cast(masks, dtype = tf.float32)
        return masks[:, tf.newaxis, tf.newaxis, :]

model_cls = ClassifyModel(tokenizer.vocab_size(), 1000, 256, 3, len(label))

In [ ]:
checkpoint = tf.train.Checkpoint(
        model=model_cls)
ckpt_manager = tf.train.CheckpointManager(
    checkpoint, '/content/drive/MyDrive/Khóa luận tốt nghiệp/Question Classify/Model_Ver1/checpoint', max_to_keep=5)
if ckpt_manager.latest_checkpoint:
    checkpoint.restore(ckpt_manager.latest_checkpoint)
    print('\nLatest checkpoint restored!!!\n')


In [ ]:
postag_label = ['B', 'Np', 'Nc', 'Nu', 'N', 'Ny', 'Ni', 'Nb', 'V', 'Vb', 'A', 'Ab', 'P', 'R', 'L', 'M', 'E', 'C', 'Cc', 'I', 'T', 'Y', 'Z', 'X', 'CH']
len(postag_label)

In [ ]:
vncore_jar = '/content/drive/MyDrive/Khóa luận tốt nghiệp/Paragraph_Raking/Tensorflow/VnCoreNLP/VnCoreNLP-1.1.1.jar'
vncore = VnCoreNLP(vncore_jar)

In [ ]:
def pos_tag(text, vncore):
    text = text.replace('_', ' ')
    tag = vncore.pos_tag(text)
    combine = []
    for i in tag:
        combine+=i
    return combine

In [ ]:
def getTypeOfWord(text):
    out = np.zeros(shape = (maxlen, len(postag_label)))
    pos = pos_tag(text, vncore)
    for i in range(len(pos)):
        if i == 277:
            print(pos)
        index_pos = postag_label.index(pos[i][1])
        out[i, index_pos] = 1
    return out

In [ ]:
class Dataset(tf.keras.utils.Sequence):
    def __init__(self, train_x, train_y, batch_size, maxlen, postag_label):
        self.train_x = train_x
        self.train_y = train_y
        self.size = len(train_x)
        self.batch_size = batch_size
        self.maxlen = maxlen
        self.postag_label = postag_label
        self.indices = np.arange(len(self.train_x))
    def __len__(self):
        return self.size//self.batch_size
    
    def __getitem__(self, idx):
        inds = self.indices[idx * self.batch_size:(idx + 1) * self.batch_size]
        x1 = np.array([tokenizer(self.train_x[i], 
                                 max_len = self.maxlen, 
                                 padding = True, 
                                 truncation = True)['input_ids'] for i in inds])
        x2 = np.array([getTypeOfWord(self.train_x[i]) for i in inds])
        y = np.array([self.train_y[i] for i in inds]).astype('float32')
        return x1, x2, y
    def on_epoch_end(self):
        np.random.shuffle(self.indices)
    

dataset_train = Dataset(train_x, train_y, 32, maxlen, postag_label)
dataset_test = Dataset(test_x, test_y, 32, maxlen, postag_label)

In [ ]:
for x1, x2, y in dataset_train:
    break
x1.shape, x2.shape, y.shape


In [ ]:
def fullConnected(d_model):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(d_model*2, activation='gelu'))
    model.add(tf.keras.layers.Dense(d_model, activation = 'sigmoid'))
    return model

class QuestionAnalys(tf.keras.Model):
    def __init__(self, d_model, maxlen, model_cls, num_heads):
        super(QuestionAnalys, self).__init__()
        self.model_cls = model_cls
        self.model_cls.trainable = False

        self.mha1 = tf.keras.layers.MultiHeadAttention(num_heads, d_model)
        self.mha2 = tf.keras.layers.MultiHeadAttention(num_heads, d_model)

        self.layerNorm1 = tf.keras.layers.LayerNormalization()
        self.layerNorm2 = tf.keras.layers.LayerNormalization()

        self.fc1 = fullConnected(d_model)
        self.fc2 = fullConnected(d_model)
        self.fc3 = fullConnected(d_model)

        self.fc_postag = tf.keras.layers.Dense(d_model)
        self.fc_type = tf.keras.layers.Dense(d_model)

        self.out1 = tf.keras.layers.Dense(1, activation = 'gelu')
        self.flatten = tf.keras.layers.Flatten()
        self.out2 = tf.keras.layers.Dense(d_model, activation = 'gelu')
        self.out3 = tf.keras.layers.Dense(maxlen)

        self.dropout1 = tf.keras.layers.Dropout(0.2)
        self.dropout2= tf.keras.layers.Dropout(0.2)

    def mask(self, inputs):
        masks = tf.logical_not(tf.math.equal(inputs, 0))
        masks = tf.cast(masks, dtype = tf.float32)
        return masks[:, tf.newaxis, tf.newaxis, :]
    
    def call(self, x):
        inputs, input_postag = x
        emb = self.model_cls.embedding(inputs)
        mask = self.mask(inputs)
        out_mha = self.mha1(emb, emb, emb, attention_mask = mask)
        norm = self.layerNorm1(out_mha+emb)
        ff = self.fc1(self.dropout1(norm))

        postag = self.fc_postag(input_postag)
        out_mha = self.mha2(ff, postag, ff, attention_mask = mask)
        norm = self.layerNorm2(ff+out_mha)
        ff = self.fc2(self.dropout2(norm))

        inputs = tf.cast(inputs, dtype = tf.int32)
        out_type = tf.expand_dims(self.fc_type(self.model_cls(inputs)), axis = 1)
        out = tf.concat([ff, out_type], axis = 1)
        out = self.fc3(out)
        out = self.out1(out)
        out = self.flatten(out)
        out = self.out2(out)
        out = self.out3(out)
        return out
    
    def compile(self, loss, optimizer):
        super(QuestionAnalys, self).compile()
        self.optimizer = optimizer
        self.loss = loss
        self.train_loss = tf.keras.metrics.Mean()
        self.test_loss = tf.keras.metrics.Mean()
        
    @property
    def metrics(self):
        return [self.train_loss, self.test_loss]

    def train_step(self, data):
        x1, x2, y = data
        with tf.GradientTape() as tape:
            predicts = self([x1, x2])
            l = self.loss(y, predicts)
        gradient = tape.gradient(l, self.trainable_variables)
        self.optimizer.apply_gradients(zip(gradient, self.trainable_variables))
        self.train_loss(l)
        return {'loss': self.train_loss.result()}
        
    def test_step(self, data):
        x1, x2, y = data
        predicts = self([x1, x2])
        l = self.loss(y, predicts)
        self.test_loss(l)
        return {'loss': self.test_loss.result()}

In [ ]:
d_model = 256
num_heads = 6
model_analys = QuestionAnalys(d_model, maxlen, model_cls, num_heads)
model_analys([x1, x2]).shape

In [ ]:
model_analys.summary()

In [ ]:
optimizer = tf.keras.optimizers.Adam(1e-4)
loss = tf.keras.losses.MeanSquaredError(reduction='none')

In [ ]:
def loss_fn(y_true, y_pred):    
    loss_value = (y_true-y_pred)**2
    mask = tf.equal(y_true, tf.zeros_like(y_true))
    mask = tf.cast(tf.logical_not(mask), dtype = tf.float32)
    loss_value *= mask
    return tf.reduce_mean(loss_value)

In [ ]:
model_analys.compile(loss = loss_fn, optimizer=optimizer)

In [ ]:
hist = model_analys.fit(dataset_train, epochs=10, validation_data=dataset_test)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(hist.history['loss'], label = 'loss')
plt.plot(hist.history['val_loss'], label = 'val loss')
plt.legend()
plt.show()

In [ ]:
checkpoint = tf.train.Checkpoint(
        model=model_analys)
ckpt_manager = tf.train.CheckpointManager(
    checkpoint, '/content/drive/MyDrive/Khóa luận tốt nghiệp/weight_of_word/chekcpoint3', max_to_keep=5)


In [ ]:
ckpt_manager.save()

In [ ]:
!pwd

In [ ]:
# model_analys.save_weights('analys.weight')

In [ ]:
def predict(text):
    ids = tokenizer(text, max_len = maxlen, padding = True, truncation = True)['input_ids']
    ids = tf.convert_to_tensor([ids], dtype = tf.int32)
    pos = getTypeOfWord(text)
    pos = tf.expand_dims(tf.convert_to_tensor(pos, dtype = tf.int32), axis = 0)
    result = model_analys([ids, pos])
    return result

In [ ]:
q = 'quy chế đào tạo theo hệ thống tín chỉ là gì'
text = vncore_tokenize(q)
result = predict(text)
for i, j in enumerate(text.split()):
    print('%s ===> %.2f'%(j, result[0, i]))

NameError: ignored

In [ ]:
ViPosTagger.postagging(ViTokenizer.tokenize(q))

In [ ]:
q = 'không biết trách nhiệm của phòng công tác sinh viên là làm gì ạ em cám ơn'
text = vncore_tokenize(q)
result = predict(text)
for i, j in enumerate(text.split()):
    print('%s ===> %.2f'%(j, result[0, i]))

In [ ]:
ViPosTagger.postagging(ViTokenizer.tokenize(q))

In [ ]:
q = 'chào admin em muốn hỏi niên giám là gì ạ'
text = vncore_tokenize(q)
result = predict(text)
for i, j in enumerate(text.split()):
    print('%s ===> %.2f'%(j, result[0, i]))

In [ ]:
ViPosTagger.postagging(ViTokenizer.tokenize(q))

In [ ]:
model_analys.save_weights('weight_of_word/model.hdf5')

In [ ]:
class WeightModule(tf.Module):
    def __init__(self, model):
        self.model = model
    def __call__(self, x):
        predict = self.model(x)
        return predict

In [ ]:
weight = WeightModule(model_analys)

In [ ]:
weight([x1, x2]).shape

In [ ]:
tf.saved_model.save(weight, 'weight_of_word/best_model')

In [ ]:
weight = tf.saved_model.load('weight_of_word/best_model')